In [ ]:
!git clone https://github.com/icomse/8th_workshop_MachineLearning.git
import os
os.chdir('8th_workshop_MachineLearning/data')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

## Data Cleaning

In [ ]:
cld=pd.read_csv('../data/HCEPDB_100K_cleaned.csv') # change where it is

In [ ]:
sns.displot(cld['pce'],kde=True)

Hmm.  Looks like those PCE entries at zero are probably errors.

In [ ]:
sns.displot(cld['jsc'],kde=True)

In [ ]:
cld = cld[cld['pce']!=0]

In [ ]:
sns.displot(cld['pce'],kde=True)

Now we should be able to make more robust predictions!

### Data imputation

There's a range of different ways to impute data in both scikit learn and pandas. The cleanest data to input is when the data is arranged in some increasing array, or in some timeseries; then one can simply interpolate the missing numbers, i.e. putting in a number that is between it's neighbors.    

However, in data set, there's not a clear ordering. 

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer

In [ ]:
cld=pd.read_csv('../data/HCEPDB_100K_cleaned.csv') # change where it is

Data inputation works on values like `np.nan`, so we will replace the 0's with `NaN`'s. `inplace` mans in the current dataframe.

In [ ]:
imp = SimpleImputer(missing_values=0.0, strategy='mean')

In [ ]:
imputed = imp.fit_transform(cld[['pce','jsc']])

In [ ]:
cld[['pce','jsc']] = imputed

In [ ]:
sns.displot(cld['pce'],kde=True)

Let's try some more complex imputation

In [ ]:
cld=pd.read_csv('../data/HCEPDB_100K_cleaned.csv') # change where it is

In [ ]:
cld.head()

In [ ]:
imp_cluster = KNNImputer(missing_values=0.0)

In [ ]:
fit_cols = ['pce','jsc','mass','e_homo_alpha','e_lumo_alpha']

In [ ]:
imp_cluster.fit(cld[fit_cols])

In [ ]:
imp_cluster.transform(cld[fit_cols])

In [ ]:
cld[fit_cols] = imp_cluster.transform(cld[fit_cols])

In [ ]:
sns.displot(cld['pce'],kde=True)

#### Another cleaning example
Let's try cleaning some real-life Excel data.

In [ ]:
# loading chlorine dataset
df_cl = pd.read_excel('../Data/DES_SurfaceTension.xlsx', sheet_name='model1 (QSPR Cl)')

In [ ]:
df_cl.head()

In [ ]:
# We can see that our preferred column headers are on the 3rd column (index no.2) of the dataframe. Let's rename our column headers.
df_cl = df_cl.rename(columns= dict(df_cl.iloc[2]))

In [ ]:
df_cl.head()

In [ ]:
# Our data starts from 3rd row, look at the No. column. Let's start from No. = 1
df_cl = df_cl[3:]
df_cl.head()

In [ ]:
# Let's use the No. column as our index
df_cl = df_cl.set_index('No.')
df_cl.head()

In [ ]:
# We see a lot of NaN values, they are basically blank cells in the excel sheet that we loaded. Let's get rid of them step by step.

# First let's aim for columns where all the values are NaN
df_cl = df_cl.dropna(axis = 1,how='all')
df_cl.head()

In [ ]:
# Now let's remove rows that have NaN values in any of it's cells. There should be no NaN in our dataframe
df_cl = df_cl.dropna(axis = 0,how='any')

In [ ]:
# Wait, there's a NaN in our column header. This column is a part of HBA, so let's add this to HBA and get rid the NaN column.
df_cl.iloc[:,0] = df_cl.iloc[:,0] + df_cl.iloc[:,1]

In [ ]:
# But how do we say get rid of NaN column? NaN is not a string. What we can do is access it through the numpy library.

df_cl = df_cl.drop(columns=[np.nan])

In [ ]:
# The dataset also contains some pre calculated descriptors, predictions, standard deviations, etc. But we only need some of them.
columns_to_keep = ['HBA', 'HBD', 'EXP. Data', 'Status', 'HBA:HBD']
df_cl.columns = [x.strip() for x in df_cl.columns] # to get rid of possible whitespaces in the column name
df_cl = df_cl[columns_to_keep]
df_cl

Now, let's load the Bromine dataset. Thankfully they are organized similarly to the clorine dataset. So, we can just reuse our previous codes.

In [ ]:
import pandas as pd
df_br = pd.read_excel('DES_SurfaceTension.xlsx', sheet_name='model2 (QSPR Br)')

In [ ]:
# Let's add these two datasets
df = pd.concat([df_cl, df_br], ignore_index = True)
df

In [ ]:
#Let's check if there are any duplicated rows
df[df.duplicated(keep=False)]

In [ ]:
#There are some, let's remove them
df=df.drop_duplicates()
df = df.reset_index()
df.head()

### additional data cleaning

In [ ]:
# Let's start with HBAs
df['HBA'] = df['HBA'].str.lower() #First, let's make them all smaller case to make them case insensitive
df['HBA'] = df['HBA'].str.split(' ').str.join('') #first, we'll get rid of all the spaces and make them one word
df['HBA'] = df['HBA'].str.replace('chloride', ' chloride').str.replace('bromide', ' bromide') # now, let's give cation and anion a space
df['HBA'] = df['HBA'].str.replace('-','') # hyphens are causing inconsistencies, let's just get rid of them
#Okay our dataset improved a lot. However, there are still some typos, but it's really difficult to find a pattern here. As it's a small dataset, we can fix them by hardcoding.
df = df.replace({'methyltriphenylphosphium bromide' : 'methyltriphenylphosphonium bromide',                 # Hardcoding to correct some typos
                 'methyltriphenylphosphunium bromide' : 'methyltriphenylphosphonium bromide',
                 'n,ndiethylethanolammonium chloride' : 'n,n-diethylethanolammonium chloride',
                  'nbutyltriphenylphosphonium bromide' : 'n-butyltriphenylphosphonium bromide'})

# we'll do similar works for HBDs
df['HBD'] = df['HBD'].str.lower()
df['HBD'] = df['HBD'].str.split(' ').str.join('')
# we'll hardcode to correct some pesky typos. Some of them like space before acid and no space before aciddihydrate may not make sense now but it's required for SMILES conversion
df['HBD'] = df['HBD'].str.replace('acid',' acid').str.replace(' aciddihydrate','aciddihydrate').replace('1,4butanediol', '1,4-butanediol')

Finally! Let's look at the distribution of HBAs and HBDs in the dataframe

In [ ]:
unique_HBA = df['HBA'].value_counts()
unique_HBD = df['HBD'].value_counts()
import matplotlib.pyplot as plt
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(16,8), dpi=200)
ax1.barh(unique_HBA.index, unique_HBA.values)
ax1.set_title("HBA distribution")
ax2.barh(unique_HBD.index, unique_HBD.values)
ax2.set_title("HBD distribution")
# We can see the repetition of some components due to representational differences and typos. Let's fix them. (Go up and uncomment those lines)